In [1]:
%pip install import-ipynb
import import_ipynb
import load_data
import clean_column_names
# import write_file_as_output
# import normalize_strings
# import fill_missing_values
# import text_to_numeric
# import drop_columns
import pandas as pd


df = load_data.load_file(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop_COPY.xlsx')
print("FILE LOADED: \n\n ", "\n")

# Copy of original data file
file_toBe_cleansed = df.copy()

# file_toBe_cleansed = load_data.load_file(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\concert_tours_by_women\concert_tours_by_women_ORIGINAL.csv')
# print("FILE LOADED: \n\n ", file_toBe_cleansed, "\n")

#cleansed_file = clean_column_names.clean_column_names(file_toBe_cleansed)
# print("CLEANSED FILE: \n" ,cleansed_file, "\n")

#cleansed_file = normalize_strings.normalize_strings(file_toBe_cleansed)
# print("CLEANSED FILE w/ strings normalized: \n" ,cleansed_file ,"\n")

# #cleansed_file = fill_missing_values.fill_missing(file_toBe_cleansed)
# print("CLEANSED FILE w/ missing values filled: \n" ,cleansed_file ,"\n")

# #cleansed_file = text_to_numeric.text_to_numeric(file_toBe_cleansed)
# print("CLEANSED FILE w/ numbers converted to numeric values: \n" ,cleansed_file ,"\n")

# cleansed_file = drop_columns.drop_columns(file_toBe_cleansed)
# print("CLEANSED FILE: COLUMNS DROPPED: \n" ,cleansed_file ,"\n")

# write_file_as_output.output_file(cleansed_file)




Note: you may need to restart the kernel to use updated packages.
FILE LOADED: 

  



In [2]:
# WRITE CLEANSED FILE AS OUTPUT
def output_file(df):
    # MAKE SURE TO LIST FULL PATH TO OUTPUT FILE OTHERWISE FILL WILL BE WRITTEIN IN DIRECTORY IN WHICH THIS FILE RESIDES!
    df.to_csv(r'C:\Users\Cessn\OneDrive\Desktop\sample_datasets\city_of_baton_rouge\cats\Bus_Stop_COPY.xlsx')
    print(f"Cleaned data saved!")

# 8 Most Common Data Cleansing Techniques:
## 1. Dropping Columns:
cols_to_drop = ["column_names_to_be_dropped"] > df_copy.drop(to_drop, axis = 1) NOTE: axis = 0 (rows) axis = 1 (columns)

In [3]:
# Drop irrelevant/ unnecessary columns
cols_to_drop = ["ROUTE NO. 1",
               'ROUTE NO. 2','ROUTE NO. 3',
               'ROUTE NO. 4','ROUTE NO. 5','ROUTE NO. 6',
                'ROUTE NO. 7','ROUTE NO. 8','ROUTE NO. 9','ROUTE NO. 10',]

file_toBe_cleansed = file_toBe_cleansed.drop(cols_to_drop, axis = 1)
output_file(file_toBe_cleansed)

Cleaned data saved!


# 2. Removing Duplicates
## Info - https://www.w3schools.com/python/pandas/ref_df_duplicated.asp

Syntax: <br>
get duplicates returned in DataFrame Format: df_copy[df_copy.duplicated()] <br>
get duplicates returned in Text Format: df_copy.duplicated()

# 3. Removing/ Dropping Irrelevant/ Unnecessary Rows
Syntax Ex: <br>
Check for Jobs located in State of Ohio in DataFrame: <br>
df_copy['Location'].str.contains("Oh", case=False) <br> <br>
df_copy['solumn_Name'].str.contains("specified_String", case=False) <br>


## 4. Removing Nulls and Blanks
# Get count of na (Not a Number values) - df_copy.isna().sum

# Get count of null (null values) - df_copy.isnull().sum
# Get count of null / na values fpr specific column - df["column name"].isna() / df['column name'].isnull()
# For multiple columns pass a list of column names in - df.[["list_of_column_names"]].isna()


# 5 - Standardizing Values (6:26)

# SYNTAX Examples: <br>
# NOTE use df.loc as shown below to get specific column info: <br>
df_copy.loc[row_indexer, ["column_name"]] <br>
df_copy.loc[0:, ["column_name"]]

In [ ]:
# NOTE use df.loc as shown below to get specific column info
# df_copy.loc[0:, ["Job Title"]]

In [ ]:
# Return all elements in Job Title column where Job Title string contains a specific word
# df_copy[df_copy["Job Title"].str.contains('scientist', case=False)]


In [ ]:
# Standardize all strings containing word scientist to read "Data Scientist"
# df_copy.loc[df_copy["Job Title"].str.contains("scientist", case=False), "Job Title"] = "Data Scientist"

In [ ]:
# Check to ensure fields in Job Title Column have been updated
# df_copy.loc[:, "Job Title"]

In [ ]:
# Capture /show all strings containing the letter k or K in column Salary Estimate 
# df_copy.loc[df_copy["Salary Estimate"].str.contains("k", case=False), "Salary Estimate"]

In [ ]:
# Replace letter "k" Salary Estimate column with 000 
# Final value: 181K - 182K > 181000 - 182000

# Use dictionay object as per below syntax examples:

# replace_dict ={"\$": "", 
#                "K": "000",
#               }
# df_copy["Salary Estimate"] = df_copy['Salary Estimate'].replace(replace_dict, regex=True)

In [ ]:
# df_copy["Salary Estimate"]

# 6 - Splitting Columns 8:16
## Documentation: https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html

![image.png](attachment:912d4e05-947c-4236-8694-750cb88270eb.png) <br>
![image.png](attachment:1ee4e5d9-66fa-4945-96a8-08d4825b0bfc.png) <br>
![image.png](attachment:8d0ad171-dbdd-4a2d-a3be-5807155377ab.png) <br>
![image.png](attachment:ecd63efd-eaf3-4dc9-8515-21feaba93a14.png) <br>